# Downstream application package

Invoke the downstream application package that calculates the mean.

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-workflows/downstream/



## Setup

In [2]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

oras pull ghcr.io/eoap/zarr-cloud-native-format/app-water-bodies-occurrence:latest --output ${WORKSPACE}

ls -l ${WORKSPACE}/app-water-bodies-occurrence.cwl

78
78
78
78
78✓ Pulled      app-water-bodies-occurrence.cwl        1.67/1.67 kB 100.00%    1ms
8  └─ sha256:c51b44b10234349ced8697402404a1e9565cf20b92434b8db4ae7748bacbf197
8✓ Pulled      application/vnd.oci.image.manifest.v1+j. 569/569  B 100.00%     0s
8  └─ sha256:23a5df24cd314c218b6f240a6724f9775c7632014348a374d8f9d2d9ea284526
8Pulled [registry] ghcr.io/eoap/zarr-cloud-native-format/app-water-bodies-occurrence:latest
Digest: sha256:23a5df24cd314c218b6f240a6724f9775c7632014348a374d8f9d2d9ea284526
-rw-rw-r-- 1 fbrito fbrito 1714 Jan 20 10:45 /workspace/zarr-cloud-native-format/app-water-bodies-occurrence.cwl


## Run the downstream application package 

Inspect and use `cwltool` to run the downstream application package CWL Workflow:


In [3]:
cat ${WORKSPACE}/app-water-bodies-occurrence.cwl | yq e '.$graph[0]' -

class: Workflow
id: water-bodies-occurrence
label: Water bodies occurrence
doc: Water bodies occurrence based on NDWI and otsu threshold
requirements: []
inputs:
  zarr-stac-catalog:
    type: Directory
    doc: Input STAC catalog with datacube
    label: Zarr store STAC Catalog
outputs:
  stac-catalog:
    type: Directory
    outputSource:
      - step_occurrence/stac-catalog
    doc: Output STAC catalog with water bodies occurrence
    label: STAC catalog
steps:
  step_occurrence:
    label: Water bodies occurrence
    doc: Water bodies occurrence based on NDWI and otsu threshold
    run: "#occurrence"
    in:
      zarr-stac-catalog: zarr-stac-catalog
    out:
      - stac-catalog


Create the job order using the outputs of notebook "2-CWl-Workflows/1-Zarr-Cloud-native-format" 

In [4]:
zarr_catalog_path=$(cat zarr-cloud-native-results.json | jq -r '.zarr_stac_catalog.path' -) 
echo $zarr_catalog_path

/workspace/zarr-cloud-native-format/runs/ja532hgm


In [5]:
cat <<EOF > downstream-params.yaml
zarr-stac-catalog: 
  class: Directory
  path: ${zarr_catalog_path}
EOF

cat downstream-params.yaml | yq .

zarr-stac-catalog:
  class: Directory
  path: /workspace/zarr-cloud-native-format/runs/ja532hgm


Use `cwltool` to run the CWL Workflow

In [6]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/app-water-bodies-occurrence.cwl#water-bodies-occurrence \
    downstream-params.yaml > downstream-results.json 2> downstream.log

Let's look at the content of the stderr:

In [7]:
cat downstream.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/app-water-bodies-occurrence.cwl#water-bodies-occurrence' to 'file:///workspace/zarr-cloud-native-format/app-water-bodies-occurrence.cwl#water-bodies-occurrence'
INFO [workflow ] start
INFO [workflow ] starting step step_occurrence
INFO [step step_occurrence] start
Error: no such object: "ghcr.io/eoap/zarr-cloud-native-format/occurrence@sha256:c19eea57eb7ddf9644655e25e08e56d446e985b96cbdddad35562ff6132e51d2"
INFO ['podman', 'pull', 'ghcr.io/eoap/zarr-cloud-native-format/occurrence@sha256:c19eea57eb7ddf9644655e25e08e56d446e985b96cbdddad35562ff6132e51d2']
Trying to pull ghcr.io/eoap/zarr-cloud-native-format/occurrence@sha256:c19eea57eb7ddf9644655e25e08e56d446e985b96cbdddad35562ff6132e51d2...
Getting image source signatures
Copying blob sha256:c494cc4848103797fced049e1227d74cef28f02ed12486fe09bf81814791111d
Copying blob sha256:50e575f7a9f613662110e4863671142eda0eca7526b7e445747650effa

Let's inspect the stdout produced. The `output` block with the id `stac-catalog` of the CWL description is a directory containing the STAC catalog:

In [8]:
cat downstream-results.json | jq '.["stac-catalog"].path' -

"/workspace/zarr-cloud-native-format/runs/kzg9lqsd"


List the STAC Catalog content:

In [9]:
stac describe $(cat downstream-results.json | jq -r '.["stac-catalog"].path' - )/catalog.json

* <Catalog id=catalog>
  * <Item id=occurrence>


In [10]:
tree $(cat downstream-results.json | jq -r '.["stac-catalog"].path' - )

/workspace/zarr-cloud-native-format/runs/kzg9lqsd
├── catalog.json
└── occurrence
    ├── occurrence.json
    └── water_bodies_mean.tif

2 directories, 3 files
